In [ ]:
!pip install openai chromadb PyPDF2

This notebook demonstrates a simple Retrieval-Augmented Generation (RAG) system. It extracts text from a PDF, chunks the text, creates embeddings using OpenAI, stores them in ChromaDB, and then uses the stored embeddings to retrieve relevant context for answering questions using an OpenAI GPT model.

In [ ]:
from openai import OpenAI
import chromadb
from PyPDF2 import PdfReader



In [15]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY').strip()
client = OpenAI(api_key=openai_api_key)



In [12]:
# Step 1: Extract text
def extract_text_from_pdf(path):
    reader = PdfReader(path)
    return " ".join([page.extract_text() for page in reader.pages])

text = extract_text_from_pdf("PRD.pdf")



In [13]:
# Step 2: Chunk text
def chunk_text(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

chunks = chunk_text(text)



In [16]:
# Step 3: Create embeddings
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="docs")

for i, chunk in enumerate(chunks):
    embedding = client.embeddings.create(
        model="text-embedding-3-small",
        input=chunk
    ).data[0].embedding
    collection.add(documents=[chunk], embeddings=[embedding], ids=[str(i)])

In [17]:
# Step 4: Query + Retrieval
def query_doc(question):
    q_embed = client.embeddings.create(
        model="text-embedding-3-small", input=question
    ).data[0].embedding

    results = collection.query(query_embeddings=[q_embed], n_results=3)
    context = " ".join(results["documents"][0])

    # Step 5: Ask GPT
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Use the provided context to answer factually."},
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {question}"}
        ]
    )
    return response.choices[0].message.content

print(query_doc("What is the key takeaway from this paper?"))

The key takeaway from the paper is the implementation of a robust evaluation and cost optimization strategy for enhancing the performance of a large language model (LLM) search system. The strategy includes carefully testing changes on a small portion of traffic, utilizing multiple models (like GPT-4 and GPT-3.5) for different query complexities, and employing techniques such as response caching, prompt optimization, and offline batch evaluation to significantly reduce operating costs while maintaining high quality and reliability in responses. This ensures that the system continuously improves based on defined metrics and user outcomes, ultimately providing value to stakeholders while managing financial resources effectively.
